In [1]:
# Import Required Packages
import torch
import os
import json
import sys
import re
import random
import importlib.util
from typing import *
from tqdm import tqdm 
from typing import List
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
import torch as nn

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification,Trainer, TrainingArguments, PreTrainedModel,AutoConfig,DataCollatorWithPadding
from peft import get_peft_model, LoraConfig
from datasets import Dataset

import gc


from JS_Architects import *

/home/jdelinea/anaconda3/envs/Env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Summary

We will use Architects model

1) Implement Custom pipeline, no prompting just prediction with shrekking of the embedding 

2) Automated Forsequence Classification
    1) No prompting
    2) Prompt: "Classify with labels encode in binary"
    3) Prompt: "Clasify with list label: ["depth",...]"
    4) Prompt "Classify with list label: ["depth",...]. Depth represents..., Containment represents...."

# Custom Pipeline 

## 1. Prepare Data

### Load 


In [2]:
with open("perceptions_training.json", "r") as f:
    dic_training = json.load(f)

with open("perceptions_training_full.json", "r") as f:
    dic_training_full = json.load(f)

with open("perceptions_testing.json", "r") as f:
    dic_testing = json.load(f)

label_list = [
    "containment",
    "depth",
    "symmetry",
    "categorical",
    "spatial-Orientation",
    "spatial-Ordinal",
    "similarity",
    "quantitative",
    "replication",
    "figure-Ground",
    "continuity",
    "size",
    "closure",
    "centroid",
    "topological",
    "motion",
]

### Format

In [3]:
def prepare_data_for_multilabel_classification(
    dic_training,
    instruction,
    label_list,
    inp_prefix="<I>",
    out_prefix="<O>",
    arr_sep="\n",
    exa_sep="\n---\n",
    bos_token="<|begin_of_text|>",
    eos_token="<|end_of_text|>"
):
    llama_data = []

    # Create a mapping from label to index
    label_to_index = {label: i for i, label in enumerate(label_list)}

    for entry_id, content in dic_training.items():
        # Extract perceptions (labels) and encode them as a binary vector
        perceptions = content.get("perceptions", [])
        label_vector = labels_to_binary(label_list, perceptions)

        # Combine train and test examples
        examples = content.get("example", {}).get("train", []) + content.get("example", {}).get("test", [])

        # Format examples into a single input string
        formatted_examples = []
        for example in examples:
            input_data = f"{inp_prefix}{format_array(example['input'], arr_sep)}"
            output_data = f"{out_prefix}{format_array(example['output'], arr_sep)}{eos_token}"
            formatted_examples.append(f"{input_data}{exa_sep}{output_data}")

        # Combine all examples into one input text and prepend the BOS token
        combined_text = f"{exa_sep.join(formatted_examples)}"

        # Add the structured data for fine-tuning
        llama_data.append({
            "instruction": f"{instruction}",
            "input": combined_text,
            "output": label_vector,  # Multi-label as binary vector
        })

    return llama_data

def format_array(array, arr_sep="\n"):
    """
    Helper function to format a 2D array into a string with row-wise separation.
    """
    return arr_sep.join([" ".join(map(str, row)) for row in array])

def labels_to_binary(label_list, input_labels):
    """
    Convert perceptions into a binary vector based on the label list.
    Handles both single strings and lists of strings for input_labels.
    """
    # Ensure input_labels is treated as a list
    if isinstance(input_labels, str):
        input_labels = [input_labels]
    
    # Create a set of lowercase input labels
    input_set = set(label.lower() for label in input_labels)
    
    # Generate the binary vector
    return [1 if label.lower() in input_set else 0 for label in label_list]


In [4]:
instruction = (
    "Classify the sequence of input-output pairs based on the following categories: "
    + ", ".join(label_list)
    + "."
)

llama_data = prepare_data_for_multilabel_classification(
    dic_training, # dic_training_full
    instruction,
    label_list)

llama_data_list_testing = prepare_data_for_multilabel_classification(
    dic_testing,
    instruction,
      label_list)

# Dict
llama_data_dict = {
    "instruction": [item["instruction"] for item in llama_data],
    "input": [item["input"] for item in llama_data],
    "output": [item["output"] for item in llama_data],
}

# Restructure llama_data
llama_data_dict_testing = {
    "instruction": [item["instruction"] for item in llama_data_list_testing],
    "input": [item["input"] for item in llama_data_list_testing],
    "output": [item["output"] for item in llama_data_list_testing],
}

# Dataset
llama_data_dataset = Dataset.from_dict(llama_data_dict)
llama_data_dataset_testing = Dataset.from_dict(llama_data_dict_testing)

llama_data_dataset_testing[0]

{'instruction': 'Classify the sequence of input-output pairs based on the following categories: containment, depth, symmetry, categorical, spatial-Orientation, spatial-Ordinal, similarity, quantitative, replication, figure-Ground, continuity, size, closure, centroid, topological, motion.',
 'input': '<I>8 6\n6 4\n---\n<O>8 6 8 6 8 6\n6 4 6 4 6 4\n6 8 6 8 6 8\n4 6 4 6 4 6\n8 6 8 6 8 6\n6 4 6 4 6 4<|end_of_text|>\n---\n<I>7 9\n4 3\n---\n<O>7 9 7 9 7 9\n4 3 4 3 4 3\n9 7 9 7 9 7\n3 4 3 4 3 4\n7 9 7 9 7 9\n4 3 4 3 4 3<|end_of_text|>\n---\n<I>3 2\n7 8\n---\n<O>3 2 3 2 3 2\n7 8 7 8 7 8\n2 3 2 3 2 3\n8 7 8 7 8 7\n3 2 3 2 3 2\n7 8 7 8 7 8<|end_of_text|>',
 'output': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]}

## 2. Load Model and Tokenizer

Since we run out of memory we used 3B parameter model

In [5]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B"
                                             #,torch_dtype=torch.float16
                                             )

print(len(tokenizer.vocab))

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.35s/it]

128256


## 3. Shrink the tokenizer and embedding 

In [6]:
special_tokens_dict = {
    "input": "<I>",
    "output": "<O>",
    "array_sep": "\n",
    "example_sep": "\n---\n",
    "eos_token": "<|end_of_text|>",
    "bos_token": "<|begin_of_text|>",
    "pad_token": "[PAD]"
}

# Add special tokens
tokenizer.add_special_tokens({
    "additional_special_tokens": [
        special_tokens_dict["input"],
        special_tokens_dict["output"],
        special_tokens_dict["array_sep"],
        special_tokens_dict["example_sep"]
    ],
    "eos_token": special_tokens_dict["eos_token"],
    "bos_token": special_tokens_dict["bos_token"],
    "pad_token": special_tokens_dict["pad_token"]
})

# Set the tokenizer pad token explicitly
tokenizer.pad_token = special_tokens_dict["pad_token"]

# Check the updated tokens
print(f"Special Tokens: {tokenizer.special_tokens_map}")
print(f"Vocabulary Size: {len(tokenizer)}")

# Resize model embeddings
model.resize_token_embeddings(len(tokenizer))


Special Tokens: {'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '[PAD]', 'additional_special_tokens': ['<I>', '<O>', '\n', '\n---\n']}
Vocabulary Size: 128261


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128261, 3072)

In [7]:
def build_corpus_for_shrinking(hf_dataset):
    """
    Concatenate the 'input' + 'instruction' from the dataset
    to ensure all relevant tokens appear.
    """
    corpus_list = []
    for sample in hf_dataset:
        text = (sample["instruction"] or "") + " " + (sample["input"] or "")
        corpus_list.append(text)
    # Combine into one big string
    corpus = "\n".join(corpus_list)
    return corpus

corpus = build_corpus_for_shrinking(llama_data_dataset)

len(corpus)

423226

shrink_embeddings(
            model=model,
            tokenizer=tokenizer,
            corpus=corpus,                  # ensures relevant tokens are kept
            keep_special_tokens=True,
            keep_normalizer=False,
            keep_token_order=True
        )

print("Tokenizer size after shrinking:", len(tokenizer.vocab))

tokenizer.vocab

In [8]:
test_sequence = llama_data_dataset[0]["input"]

encoded = tokenizer.encode(test_sequence)
decoded = tokenizer.decode(encoded)
print("Encoded:", encoded)
print("Decoded:", decoded)

Encoded: [128000, 128256, 15, 220, 22, 220, 22, 128258, 22, 220, 22, 220, 22, 128258, 15, 220, 22, 220, 22, 128259, 128257, 15, 220, 15, 220, 15, 220, 15, 220, 22, 220, 22, 220, 15, 220, 22, 220, 22, 128258, 15, 220, 15, 220, 15, 220, 22, 220, 22, 220, 22, 220, 22, 220, 22, 220, 22, 128258, 15, 220, 15, 220, 15, 220, 15, 220, 22, 220, 22, 220, 15, 220, 22, 220, 22, 128258, 15, 220, 22, 220, 22, 220, 15, 220, 22, 220, 22, 220, 15, 220, 22, 220, 22, 128258, 22, 220, 22, 220, 22, 220, 22, 220, 22, 220, 22, 220, 22, 220, 22, 220, 22, 128258, 15, 220, 22, 220, 22, 220, 15, 220, 22, 220, 22, 220, 15, 220, 22, 220, 22, 128258, 15, 220, 15, 220, 15, 220, 15, 220, 22, 220, 22, 220, 15, 220, 22, 220, 22, 128258, 15, 220, 15, 220, 15, 220, 22, 220, 22, 220, 22, 220, 22, 220, 22, 220, 22, 128258, 15, 220, 15, 220, 15, 220, 15, 220, 22, 220, 22, 220, 15, 220, 22, 220, 22, 128001, 128259, 128256, 19, 220, 15, 220, 19, 128258, 15, 220, 15, 220, 15, 128258, 15, 220, 19, 220, 15, 128259, 128257, 19, 22

## 4. Apply LoRA to the Shrunk Model

In [9]:
# Configure LoRA
peft_config = LoraConfig(
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head"],
    lora_alpha=16,
    lora_dropout=0.0,
    bias="none"
)


# Wrap the shrunk model with LoRA
model_shrinked = get_peft_model(model, peft_config)

# Make sure to unfreeze embeddings if you want to train them directly 
# (LoRA on embed_tokens will still add ranks; but if you want the base embedding 
#  weights to be trainable, do something like):
for param in model_shrinked.get_input_embeddings().parameters():
    param.requires_grad = True


/home/jdelinea/anaconda3/envs/Env/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:543: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


## 5. Full architecture

### Add Classification head

In [10]:
from copy import deepcopy

class LLMWithClassificationHead(PreTrainedModel):
    def __init__(self, base_model, config, num_labels):
        super().__init__(config)
        if isinstance(base_model, LLMWithClassificationHead):
            raise ValueError("base_model cannot be an instance of LLMWithClassificationHead")
        
        self.base_model_1 = (base_model)

        self.num_labels = num_labels
        hidden_size = config.hidden_size

        # Ensure classifier supports fp16
        self.classifier = nn.Linear(hidden_size, num_labels)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(self, input_ids, output_hidden_states=True, return_dict=True, attention_mask=None, labels=None):
        outputs = self.base_model_1(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )
        last_hidden = outputs.hidden_states[-1]
        pooled = last_hidden[:, -1, :]  # Taking the last token's hidden state
        logits = self.classifier(pooled)

        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.float())

        return {
            "logits": logits,
            "loss": loss,
            "hidden_states": outputs.hidden_states,
        }


In [11]:
import torch
import torch.nn as nn

# Load configuration and base model
base_config = AutoConfig.from_pretrained("meta-llama/Llama-3.2-3B")

# Define number of labels for classification
num_labels = 16

# Instantiate the custom model
model_classification = LLMWithClassificationHead(
    base_model=model_shrinked,
    config=base_config,
    num_labels=num_labels
).to("cuda")


### Tokenize data

In [12]:
lengths = [len(tokenizer.encode(example["input"])) for example in llama_data_dataset]
print("Max length:", max(lengths), "95th percentile:", np.percentile(lengths, 95))


Max length: 18015 95th percentile: 7011.749999999998


In [13]:
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["input"],
        padding="longest",
        truncation=True,
        max_length=7000  # Reduced from 8192 to 2048
    )
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": examples["output"]
    }


# Keep only the input output no instructions for now
final_dataset = llama_data_dataset.remove_columns(["instruction"])
final_dataset_testing = llama_data_dataset_testing.remove_columns(["instruction"])

tokenized_final_dataset = final_dataset.map(tokenize_function, batched=True)
tokenized_final_test = final_dataset_testing.map(tokenize_function, batched=True)

tokenized_final_dataset

Map: 100%|██████████| 50/50 [00:00<00:00, 442.74 examples/s]


Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 150
})

## 6. Train

In [14]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Extract labels
        labels = inputs.pop("labels", None)
        if labels is None:
            raise ValueError("Labels are missing in inputs")

        
        # Forward pass with additional arguments
        outputs = model(**inputs, labels=labels)
        
        # Extract loss
        loss = outputs["loss"]
        
        # Return loss and outputs if required
        return (loss, outputs) if return_outputs else loss


### Training

In [15]:
# Clear cache before training
gc.collect()
torch.cuda.empty_cache()

# Define TrainingArguments with optimizations
training_args = TrainingArguments(
    output_dir="./JS_finetuned_model",
    evaluation_strategy="no",
    learning_rate=1e-4,  # instead of 1e-4
    per_device_train_batch_size=1,  # Reduced batch size
    per_device_eval_batch_size=1,  # Reduced batch size
    num_train_epochs=5,  # Increased epochs if feasible
    fp16=True,
    fp16_full_eval=True,
    gradient_accumulation_steps=4,  # Increased gradient accumulation
    #gradient_checkpointing=True,  # Enable gradient checkpointing
    #save_total_limit=1,
    #save_strategy="epoch",
    #save_steps=500,
    logging_dir="./logs",
    logging_steps=100,
    report_to="wandb",  # Log to W&B
    # Add any other necessary arguments
)

# Use built-in data collator with dynamic padding
data_collator = DataCollatorWithPadding(tokenizer, padding='longest')

# Initialize Trainer
trainer = CustomTrainer(
    model=model_classification,
    args=training_args,
    train_dataset=tokenized_final_dataset,
    #eval_dataset=tokenized_final_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start Training
#trainer.train()

# Clear memory after training
gc.collect()
torch.cuda.empty_cache()

#torch.save(model.state_dict(), "./JS_finetuned_model/pytorch_model.bin")
#model.config.save_pretrained("./JS_finetuned_model")
#tokenizer.save_pretrained("./JS_finetuned_model")

/home/jdelinea/anaconda3/envs/Env/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_751386/1697175700.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [16]:
print(f"Memory allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
print(f"Memory cached: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

Memory allocated: 12693.37 MB
Memory cached: 12742.00 MB


In [17]:
# Force garbage collection to remove unreferenced objects
gc.collect()

# List all objects and their sizes
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(f"Type: {type(obj)}, Size: {sys.getsizeof(obj)} bytes, Shape: {obj.size()}")
    except Exception as e:
        pass


/home/jdelinea/anaconda3/envs/Env/lib/python3.10/site-packages/torch/__init__.py:836: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)
/tmp/ipykernel_751386/3683129824.py:7: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):


Type: <class 'torch.Tensor'>, Size: 80 bytes, Shape: torch.Size([])
Type: <class 'torch.Tensor'>, Size: 80 bytes, Shape: torch.Size([])
Type: <class 'torch.nn.parameter.Parameter'>, Size: 80 bytes, Shape: torch.Size([128261, 3072])
Type: <class 'torch.Tensor'>, Size: 80 bytes, Shape: torch.Size([64])
Type: <class 'torch.nn.parameter.Parameter'>, Size: 80 bytes, Shape: torch.Size([3072])
Type: <class 'torch.nn.parameter.Parameter'>, Size: 80 bytes, Shape: torch.Size([3072])
Type: <class 'torch.nn.parameter.Parameter'>, Size: 80 bytes, Shape: torch.Size([3072])
Type: <class 'torch.nn.parameter.Parameter'>, Size: 80 bytes, Shape: torch.Size([3072])
Type: <class 'torch.nn.parameter.Parameter'>, Size: 80 bytes, Shape: torch.Size([3072])
Type: <class 'torch.Tensor'>, Size: 80 bytes, Shape: torch.Size([64])
Type: <class 'torch.Tensor'>, Size: 80 bytes, Shape: torch.Size([64])
Type: <class 'torch.nn.parameter.Parameter'>, Size: 80 bytes, Shape: torch.Size([3072])
Type: <class 'torch.nn.paramet

In [18]:
gc.collect()
torch.cuda.empty_cache()

print(torch.cuda.memory_summary(device=0, abbreviated=True))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  12693 MiB |  12693 MiB |  12693 MiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |  12693 MiB |  12693 MiB |  12693 MiB |      0 B   |
|---------------------------------------------------------------------------|
| Requested memory      |  12691 MiB |  12691 MiB |  12691 MiB |      0 B   |
|---------------------------------------------------------------

In [19]:
import gc
import torch

# Clear memory after training
gc.collect()
torch.cuda.empty_cache()

# Batch size
batch_size = 10
num_batches = len(tokenized_final_test) // batch_size  # Number of batches

# Run predictions on the evaluation dataset in fixed batches
predictions = []
for batch_idx in range(num_batches):
    # Slice the dataset for the current batch
    start_idx = batch_idx * batch_size
    end_idx = start_idx + batch_size
    batch = tokenized_final_dataset.select(range(start_idx, end_idx))
    
    # Run prediction for the batch
    prediction = trainer.predict(batch)
    predictions.append(prediction)
    
    # Clear memory after each batch
    gc.collect()
    torch.cuda.empty_cache()

# Combine batch predictions
logits = [pred.predictions for pred in predictions]  # Model outputs
true_labels = [pred.label_ids for pred in predictions]  # True labels
metrics = [pred.metrics for pred in predictions]  # Metrics

# Print combined metrics
# (Optional: Aggregate metrics if necessary, e.g., averaging)
print("Evaluation Metrics:", metrics)

# Clear memory after evaluation
gc.collect()
torch.cuda.empty_cache()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jean-sebastien-delineau (drykx). Use `wandb login --relogin` to force relogin


Evaluation Metrics: [{'test_loss': 0.684650182723999, 'test_model_preparation_time': 0.0138, 'test_runtime': 15.5665, 'test_samples_per_second': 0.642, 'test_steps_per_second': 0.642}, {'test_loss': 0.6784305572509766, 'test_model_preparation_time': 0.0138, 'test_runtime': 15.4493, 'test_samples_per_second': 0.647, 'test_steps_per_second': 0.647}, {'test_loss': 0.6686240434646606, 'test_model_preparation_time': 0.0138, 'test_runtime': 15.6992, 'test_samples_per_second': 0.637, 'test_steps_per_second': 0.637}, {'test_loss': 0.653450608253479, 'test_model_preparation_time': 0.0138, 'test_runtime': 16.197, 'test_samples_per_second': 0.617, 'test_steps_per_second': 0.617}, {'test_loss': 0.6514352560043335, 'test_model_preparation_time': 0.0138, 'test_runtime': 16.8, 'test_samples_per_second': 0.595, 'test_steps_per_second': 0.595}]


In [20]:
import numpy as np
from sklearn.metrics import accuracy_score, hamming_loss, precision_score, recall_score, f1_score

# Parameters
threshold = 0.1

# Placeholder for empirical labels
emp_labels = []

# Process logits
for j, batch_logits in enumerate(logits):
    exp_scores = np.exp(batch_logits[0])  # Vectorized exponentiation
    probabilities = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)  # Softmax
    pred_label_indices = np.argsort(probabilities, axis=1)[:, -2:]  # Top 2 indices
    batch_labels = (probabilities > threshold).astype(int)  # Thresholding
    emp_labels.extend(batch_labels.tolist())  # Append predictions
    
    # Logging predictions
    for i, indices in enumerate(pred_label_indices):
        example_id = i + 10 * j
        print(f"Example {example_id}")
        print("Predicted Labels:", [label_list[idx] for idx in indices])
        true_indices = [idx for idx, val in enumerate(true_labels[j][i]) if val == 1]
        print("True Labels:", [label_list[idx] for idx in true_indices])

Example 0
Predicted Labels: ['quantitative', 'motion']
True Labels: ['spatial-Ordinal', 'replication']
Example 1
Predicted Labels: ['motion', 'size']
True Labels: ['centroid']
Example 2
Predicted Labels: ['centroid', 'quantitative']
True Labels: ['replication']
Example 3
Predicted Labels: ['size', 'quantitative']
True Labels: ['motion']
Example 4
Predicted Labels: ['closure', 'size']
True Labels: ['replication', 'continuity']
Example 5
Predicted Labels: ['closure', 'quantitative']
True Labels: ['categorical']
Example 6
Predicted Labels: ['motion', 'quantitative']
True Labels: ['replication']
Example 7
Predicted Labels: ['motion', 'quantitative']
True Labels: ['categorical', 'motion']
Example 8
Predicted Labels: ['motion', 'size']
True Labels: ['topological']
Example 9
Predicted Labels: ['size', 'quantitative']
True Labels: ['size']
Example 10
Predicted Labels: ['quantitative', 'size']
True Labels: ['similarity']
Example 11
Predicted Labels: ['quantitative', 'size']
True Labels: ['repli

In [21]:
# Convert true labels to a single array
true_labels_list = np.vstack(true_labels)

# Metrics computation
y_pred = np.array(emp_labels)
y_true = true_labels_list

print('Exact Match Ratio:', accuracy_score(y_true, y_pred))
print('Hamming Loss:', hamming_loss(y_true, y_pred))
print('Precision:', precision_score(y_true, y_pred, average='samples'))
print('Recall:', recall_score(y_true, y_pred, average='samples'))
print('F1 Measure:', f1_score(y_true, y_pred, average='samples'))

Exact Match Ratio: 0.0
Hamming Loss: 0.2425
Precision: 0.06966666666666667
Recall: 0.17
F1 Measure: 0.09676190476190476
